In [7]:
import os
from skimage import io
import numpy as np
import matplotlib.pyplot as plt
import re
from skimage import transform
from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer

In [8]:
seed = 1
rng = np.random.RandomState(seed)

#mlb=MultiLabelBinarizer()

# # read data
def load_images_from_folder(folder):
    X = []
    y = []
    for filename in os.listdir(folder):
        img = io.imread(os.path.join(folder,filename))
        if img is not None:
            label = re.findall(r'_([^.]*).',filename)
            X.append(img)
            y.append(int(label[0]))
    return X,y

train = np.array(load_images_from_folder('train937'))
X_train = train[0]
y_train = train[1]
y_train=y_train.astype('int')

test = np.array(load_images_from_folder('valid516'))

In [9]:
X_train_resized = [transform.resize(img,(50,50,3)) for img in X_train]

X_train_flat = np.array([img.flatten() for img in X_train_resized])

X_test = test[0]
y_test = test[1]
y_test = y_test.astype('int')


X_test_resized = [transform.resize(img,(50,50,3)) for img in X_test]
X_test_flat = np.array([img.flatten() for img in X_test_resized])

/usr/local/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [10]:
ipca=IncrementalPCA(n_components=300, batch_size=None,whiten=True)
n_components = ipca.n_components
ipca.fit(X_train_flat)
X_train_flat=ipca.transform(X_train_flat)
X_test_flat=ipca.transform(X_test_flat)

In [14]:
x_train=X_train_flat
x_test=X_test_flat
#y_train=y_train-1
#y_test=y_test-1

def one_hot_enc(label, n_labels):
    one_hot = np.zeros(n_labels)
    one_hot[label-1] = 1
    return one_hot

#all_labels=[]
#for i in range(1,129):
#    all_labels.append(i)
#mlb.fit(all_labels)
#y_train=mlb.transform(y_train)
print(y_train.shape)
#y_test=mlb.transform(y_test)
print(y_test.shape)
y_train = np.array([one_hot_enc(label, 128) for label in y_train])
y_test = np.array([one_hot_enc(label, 128) for label in y_test])

(937,)
(516,)


In [36]:
a = np.array([0,0,1,0])
b = np.array([0,0,1,0])
print(np.array_equal(a, b))

print(np.where(a==1)[0][0])
                  
c = np.array([[1,2], [3,4]])
c = c + 1
print(c)

True
2
[[2 3]
 [4 5]]


In [33]:
### Tensorflow Neural Network
# set number of neurons in hidden layer M
M = 300

# Initialize placeholders
x = tf.placeholder(dtype = tf.float32, shape = [None, n_components])
y = tf.placeholder(dtype = tf.int32, shape = [None,128])

W1 = tf.Variable(tf.random_normal([M, M], stddev=0.03), name='W1')
b1 = tf.Variable(tf.random_normal([M]), name='b1')
# and the weights connecting the hidden layer to the output layer
W2 = tf.Variable(tf.random_normal([M, 128], stddev=0.03), name='W2')
b2 = tf.Variable(tf.random_normal([128]), name='b2')

# calculate the output of the hidden layer
hidden_out = tf.add(tf.matmul(x, W1), b1)
hidden_out = tf.nn.relu(hidden_out)

logits = tf.nn.softmax(tf.add(tf.matmul(hidden_out, W2), b2))

# maybe add some clipped shit here

# Define a loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y,
                                                                    logits = logits))
# Define an optimizer
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

# Convert logits to label indexes
correct_pred = tf.argmax(logits, 1)

# Define an accuracy metric
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

tf.set_random_seed(1234)
sess = tf.Session()

sess.run(tf.global_variables_initializer())

for i in range(2001):
        _, accuracy_val = sess.run([train_op, accuracy], feed_dict={x: x_train, y: y_train})
        if i % 500 == 0:
            print('EPOCH', i)
            #print("Loss: ", loss)
        #print('DONE WITH EPOCH')
            #print(accuracy_val)

predicted = sess.run([correct_pred], feed_dict={x: x_test})[0]

EPOCH 0
EPOCH 500
EPOCH 1000
EPOCH 1500
EPOCH 2000


In [42]:
# Tensorboard
with tf.Session() as sess:
    writer = tf.summary.FileWriter("/tmp/tb/testneutral", sess.graph)

In [38]:
def one_hot_dec(one_hot_arr):
    index = np.where(one_hot_arr==1)[0][0]
    label = index+1
    return label

In [39]:
# 0-1 loss is based on match count
print("y_test")
print(y_test)
print("predicted")
print(predicted)

# y_test is an array of one-hot arrays, while y_pred is an array of labels, thus we decode y_test
y_test = np.array([one_hot_dec(one_hot) for one_hot in y_test])

# the Neural network is written to output with 0-offset so we need to adjust "predicted"
predicted = predicted + 1

print("y_test")
print(y_test)
print("predicted")
print(predicted)

match_count = sum([int(np.array_equal(y_true, y_pred)) for y_true, y_pred in zip(y_test, predicted)])
# Calculate the accuracy
accuracy = match_count / len(y_test)

# Print the accuracy
print("Accuracy: {:.3f}".format(accuracy))

y_test
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
predicted
[ 80  25 112  71 117  51 121 112  25  67  25 106  32  80  15  30  36  68
 110  91  25  63  25  56 106  75  98 116 112   4  32  87  35  70  32 106
 126  78  32  43   6  59  69  48 117 126  35  32  43  80  92  97 106  35
  31  93  69  63  25  80  59  59 112  75 108  99  93  80  31  32  10   3
  73  93  94  68 112  69  56  30  68  27  83  51  32  32  25  78  19  56
  72  25  32  32  51  31  68  71  31  80  53  69 112   4  59  87  48 111
  10 106  13 115   3  43  83  39  27  68 118  35 117  68  98  25  77  32
 106   2  20  10  22  36  93  36  93  68  43   6  43  25 127  68  43  22
  20  32  79  48  36  93  95   5 127  67  86  32  75  71  71  80  20 117
  30  32 124  63 113  27   1  98  36  43  30  68  62  87  62 110   9  22
 122  60  25 112  97  75 106   9   6  79  93  54  98  48  59  32  83 106
  79  98  25  91 122  88